In [ ]:
!pip install transformers==4.29.2 torch==1.13.1 scikit-learn==1.2.2

In [ ]:
from google.colab import drive
!git clone https://github.com/fabianagoes/ismb_tutorial8.git
%cd ismb_tutorial8
drive.mount('/content/drive')
import torch
from transformers import AutoTokenizer, AutoModel
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
from utils import get_all_subdirs
import ipywidgets as widgets
from IPython.display import display

In [ ]:
root_dir = "/content/ismb_tutorial8/datasets"
subdirs = get_all_subdirs(root_dir) # internal function that goes over all subdirectories. Source code: utils.py

dropdown = widgets.Dropdown(
    options=subdirs,
    description='Select:',
    disabled=False,
)

selected_path = {'value': subdirs[0]}

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        selected_path['value'] = change['new']

dropdown.observe(on_change)

display(dropdown)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True)
model = AutoModel.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True)
model.eval()

In [ ]:
# Generate embeddings
def get_embedding_cls(sequence):
    inputs = tokenizer(sequence, return_tensors="pt", padding=True, truncation=True)
    inputs = {k: v.cuda() for k, v in inputs.items()}  # GPU
    with torch.no_grad():
        outputs = model(**inputs)
    hidden_states = outputs.last_hidden_state  # [batch_size, seq_len, hidden_size]
    cls_embedding = hidden_states[:, 0, :]  # Pega o token CLS (posição 0)
    return cls_embedding.cpu().numpy().flatten()

In [ ]:
# Generate embeddings for the datasets
def embed_dataframe(df):
    X = []
    for seq in df['sequence']:
        X.append(get_embedding_cls(seq))
    X = np.vstack(X)
    y = df['label'].values
    return X, y

In [ ]:
print('Dataset Selected: ',selected_path['value'])

train_data=pd.read_csv(selected_path['value']+'/train.csv')#read_csv to read csv files
val_data=pd.read_csv(selected_path['value']+'/dev.csv')
test_data=pd.read_csv(selected_path['value']+'/test.csv')
data={'Train':train_data,'Val':val_data,'Test':test_data}

train_data.head(5)# displays first 5 datapoints

X_train, y_train = embed_dataframe(train_data)
X_val, y_val = embed_dataframe(val_data)
X_test, y_test = embed_dataframe(test_data)

print(f"Train shape: {X_train.shape}")
print(f"Test shape: {X_test.shape}")

In [ ]:
# Train MLP
clf = MLPClassifier(hidden_layer_sizes=(256,), max_iter=500, random_state=42)
clf.fit(X_train, y_train)

In [ ]:
# Evaluating on test data
y_pred = clf.predict(X_test)

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

acc = accuracy_score(y_test, y_pred)
prec, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted')

print(f"Accuracy: {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")